## This is code for finding optimal L1 and L2 coefficients to improve learning on adversarial images.

In [47]:
#importing
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
import pickle as cPickle
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.use('Agg')
#from pprint import pprint

/Users/Deepak/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [48]:
#Definitions
# Convert into correct type for theano
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

# Weights are shared theano variables
def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

# RMSProp to update weights
def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

# Dropout regularization 
def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

# Neural network model, 3 fully connected layers
def model(X, w_h, w_h2, w_o, p_drop_input, p_drop_hidden):
	# Input layer: dropout + relu 
    X = dropout(X, p_drop_input)
    h = T.nnet.relu(T.dot(X, w_h))

	# Hidden layer: dropout + relu 
    h = dropout(h, p_drop_hidden)
    h2 = T.nnet.relu(T.dot(h, w_h2))
	
	# Output layer: dropout + softmax 
    h2 = dropout(h2, p_drop_hidden)
    py_x = T.nnet.softmax(T.dot(h2, w_o))
    return h, h2, py_x

def one_hot(x,n):
	if type(x) == list:
		x = np.array(x)
	x = x.flatten()
	o_h = np.zeros((len(x),n))
	o_h[np.arange(len(x)),x] = 1
	return o_h

def mnist(ntrain=60000,ntest=10000,onehot=True):
	data_dir = os.path.join(datasets_dir,'mnist/')
	fd = open(os.path.join(data_dir,'train-images-idx3-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	trX = loaded[16:].reshape((60000,28*28)).astype(float)

	fd = open(os.path.join(data_dir,'train-labels-idx1-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	trY = loaded[8:].reshape((60000))

	fd = open(os.path.join(data_dir,'t10k-images-idx3-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	teX = loaded[16:].reshape((10000,28*28)).astype(float)

	fd = open(os.path.join(data_dir,'t10k-labels-idx1-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	teY = loaded[8:].reshape((10000))

	trX = trX/255.
	teX = teX/255.

	trX = trX[:ntrain]
	trY = trY[:ntrain]

	teX = teX[:ntest]
	teY = teY[:ntest]

	if onehot:
		trY = one_hot(trY, 10)
		teY = one_hot(teY, 10)
	else:
		trY = np.asarray(trY)
		teY = np.asarray(teY)

	return trX,teX,trY,teY

def plot_mnist_digit(image1, image2, name1, name2):
    global count_attack
    image1 = np.reshape(image1,[1,784])
    image2 = np.reshape(image2,[1,784])
    #print 'test image confidence' , np.mean(predict_conf(image1)), 'adversarial image confidence', np.mean(predict_conf(image2))
    if (predict(image1) != predict(image2)):
        count_attack = count_attack + 1

In [49]:
#Loading MNIST data
datasets_dir = 'media/datasets/'
srng = RandomStreams()
TRAINING = True

print('MNIST data Loaded')
trX, teX, trY, teY = mnist(onehot=True)

# Initialize theano variables for X, Y, and shared variables for weights
X = T.fmatrix()
Y = T.fmatrix()

if TRAINING:
    # For training of the net, we initialize weights to random values
    w_h = init_weights((784, 625))
    w_h2 = init_weights((625, 625))
    w_o = init_weights((625, 10))
    params = [w_h, w_h2, w_o]
else:
    # To run experiments, just read weights we learned before
    print('Loading model...')
    with open('LearnedParamsL1_2.model','rb') as fp:
        params = cPickle.load(fp)
    w_h, w_h2, w_o = params

# Dropout model for training
noise_h, noise_h2, noise_py_x = model(X, w_h, w_h2, w_o, 0.2, 0.5)
# Use all-weights model for prediction
h, h2, py_x = model(X, w_h, w_h2, w_o, 0., 0.)
y_x = T.argmax(py_x, axis=1)

# To find confidence of test set use the following value of y_x
y_x1 = T.max(py_x, axis = 1)
# Define cost and update theano expressions

l1 = abs(w_h).sum() + abs(w_h2).sum() + abs(w_o).sum()
l2 = (w_h**2).sum() + (w_h2**2).sum() + (w_o**2).sum()

MNIST data Loaded


In [50]:
#l1 and l2 coefficients & Training Data
#=================== Parameters to chnge ===============================#
l1coef = [0.0,0.00001,0.0001]
l2coef = [0.0,0.00001,0.0001]
#=======================================================================#

for i in l1coef:
    for j in l2coef:
        print("l1coef = %f, l2coef = %f" %(i,j))
        cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y)) + i * l1 + j * l2
        updates = RMSprop(cost, params, lr=0.001)

        # Define train and predict theano functions
        train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)

        predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)
        predict_conf = theano.function(inputs=[X], outputs=y_x1, allow_input_downcast=True)
        print('Training MNIST data...')
        if TRAINING:
            # Train in 50 epochs
            for i in range(50):
                # Select minibatch and train
                for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
                    cost = train(trX[start:end], trY[start:end])
                # Show test set accuracy. Its cost is not used for optimization,
                # it is just to show progress.
                #print(i, ':  ', np.mean(np.argmax(teY, axis=1) == predict(teX)))
                # In each step save the learned weights
                with open('LearnedParamsL1.model','wb') as fp:
                    cPickle.dump(params,fp)
            print("Accuracy is:    ",np.mean(np.argmax(teY, axis=1) == predict(teX)))
            print("Confidence is:    ", np.mean(predict_conf(teX)))

Training MNIST data...
0 :   0.9355
1 :   0.9619
2 :   0.9728
3 :   0.9736
4 :   0.9763
5 :   0.9752
6 :   0.9773
7 :   0.9763
8 :   0.9789
9 :   0.9795
10 :   0.9771
11 :   0.9823
12 :   0.9804
13 :   0.9812
14 :   0.9815
15 :   0.9818
16 :   0.9814
17 :   0.9818
18 :   0.9803
19 :   0.9819
20 :   0.9816
21 :   0.9827
22 :   0.9818
23 :   0.9821
24 :   0.982
25 :   0.9828
26 :   0.9823
27 :   0.9821
28 :   0.983
29 :   0.9836
30 :   0.9821
31 :   0.9826
32 :   0.9819
33 :   0.983
34 :   0.9819
35 :   0.9833
36 :   0.9827
37 :   0.9835
38 :   0.9834
39 :   0.9832
40 :   0.9835
41 :   0.9827
42 :   0.9826
43 :   0.9833
44 :   0.9829
45 :   0.9826
46 :   0.9835
47 :   0.984
48 :   0.9846
49 :   0.9839
